In [4]:
import pandas as pd
from sqlalchemy import create_engine
import json
import os

## Overview

* USvideos_df is csv file US_category_df is json file, retrieved from kaggle.com, 
* A regular store of the JSON data into a DataFrame resulted in missing data as the JSON was not fully traversed. As a result an alternate approach was used to parse out those non-traversed data elements into list and applied to a dataframe,
* In order to merge the csv and JSON dataframes on category id an integer conversion was made on the id_list in the JSON dataframe. This prevented issues/errors with nt64 and object columns when attempting the merge. The dataframes were "left" joined on the csv dataframe,
* USvideos_df has category ID, US_category_df file has ID columns,
* "for" loop used to retrieve the ID from jsons dictionary, and created etag, title, id list,
* ID column from US_category_df renamed as category id to be used for merge files,
* "combined_USdata" is created as new dataframe,
* title_x and title_y columns renamed as video title and category_title, dataf frame renamed as renamed_combined_USdata.
* new renamed_combined_USdata dataframe saved as csv file in the resource folder, 
* Added in a country code of 'US' to the combined dataframe to distinguish a country of origin

In [5]:
csv_file = "Resources/USvideos.csv"
USvideos_df = pd.read_csv(csv_file)
USvideos_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [6]:
#removed three columns as a part of the clean up activities (publish_time,thumbnail_link,description)
new_USvideos_df= USvideos_df[["video_id","trending_date","title","channel_title","category_id","tags","views","likes","dislikes","comment_count","comments_disabled","ratings_disabled","video_error_or_removed"]].copy()
new_USvideos_df

,video_id,trending_date,title,channel_title,category_id,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,SHANtell martin,748374,57527,2966,15954,False,False,False
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,False,False,False
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,False,False,False
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,False,False,False
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40944,BZt0qjTWNhw,18.14.06,The Cat Who Caught the Laser,AaronsAnimals,15,"aarons animals|""aarons""|""animals""|""cat""|""cats""...",1685609,38160,1385,2657,False,False,False
40945,1h7KV2sjUWY,18.14.06,True Facts : Ant Mutualism,zefrank1,22,[none],1064798,60008,382,3936,False,False,False
40946,D6Oy4LfoqsU,18.14.06,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,24,I gave safiya nygaard a perfect hair makeover ...,1066451,48068,1032,3992,False,False,False
40947,oV0zkMe1K8s,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,1,"Black Panther|""HISHE""|""Marvel""|""Infinity War""|...",5660813,192957,2846,13088,False,False,False


In [15]:
# Load JSON
json_file = "Resources/US_category_id.json"

with open(json_file) as jsonfile:
    US_category_json = json.load(jsonfile)
    print(json.dumps(US_category_json,indent=2))
    



{
  "kind": "youtube#videoCategoryListResponse",
  "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM\"",
  "items": [
    {
      "kind": "youtube#videoCategory",
      "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ\"",
      "id": "1",
      "snippet": {
        "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
        "title": "Film & Animation",
        "assignable": true
      }
    },
    {
      "kind": "youtube#videoCategory",
      "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA\"",
      "id": "2",
      "snippet": {
        "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
        "title": "Autos & Vehicles",
        "assignable": true
      }
    },
    {
      "kind": "youtube#videoCategory",
      "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg\"",
      "id": "10",
      "snippet": {
        "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
        "title": "Music",
        "assignable": true
      }
    },
    {
      "kind

In [16]:
json_file = "Resources/US_category_id.json"
#Store JSON data into a DataFrame
US_category_df=pd.read_json(json_file)
US_category_df.head()

,kind,etag,items
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
2,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
3,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
4,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."


In [17]:
with open("./Resources/US_category_id.json") as data_file:
    data = json.load(data_file) 

    kind_list = []
    etag_list = []
    id_list = []
    title_list = []
    data_list = []

    for i in range(32):

        try:
            kind_list.append(data['items'][i]['kind'])
            etag_list.append(data['items'][i]['etag'])
            id_list.append(data['items'][i]['id'])
            title_list.append(data['items'][i]['snippet']['title'])
        except KeyError:
            kind_list.append("")
            etag_list.append("")
            id_list.append("")
            title_list.append("")
            
print("----- kind list ------")    
print(kind_list) 
print("----- etag list -------")
print(etag_list) 
print("------ id list --------")
print(id_list) 
print("------ title list ------")
print(title_list) 
print("--------finished -------")

----- kind list ------
['youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory', 'youtube#videoCategory']
----- etag list -------
['"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"', '"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"', '"m2yskBQFythfE4irbTIeOgYYfBU

In [18]:
US_category_df=US_category_df.rename(columns={"id": "category_id"})

US_category_df

,kind,etag,items
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
2,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
3,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
4,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
5,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
6,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
7,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
8,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
9,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."


In [19]:
US_category_df ["kind"] = kind_list
US_category_df ["etag"] = etag_list
US_category_df ["category_id"] = id_list
US_category_df ["title"] = title_list

print(US_category_df)

                     kind                                               etag  \
0   youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...   
1   youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...   
2   youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...   
3   youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...   
4   youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...   
5   youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/FJwVpGCVZ1yiJrqZb...   
6   youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf...   
7   youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/WmA0qYEfjWsAoyJFS...   
8   youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/EapFaGYG7K0StIXVf...   
9   youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...   
10  youtube#videoCategory  "m2yskBQFythfE4irbTIeOgYYfBU/G9LHzQmx44rX2S5ya...   
11  youtube#videoCategory  "m2yskBQFythf

In [20]:
US_category_df["category_id"] = US_category_df["category_id"].astype(int)

In [21]:
US_category_df

,kind,etag,items,category_id,title
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",1,Film & Animation
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",2,Autos & Vehicles
2,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",10,Music
3,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",15,Pets & Animals
4,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",17,Sports
5,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/FJwVpGCVZ1yiJrqZb...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",18,Short Movies
6,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",19,Travel & Events
7,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/WmA0qYEfjWsAoyJFS...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",20,Gaming
8,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/EapFaGYG7K0StIXVf...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",21,Videoblogging
9,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",22,People & Blogs


In [22]:
#merged on category id using left join 
combined_USdata=pd.merge(new_USvideos_df,US_category_df, on= "category_id", how = "left")
combined_USdata

,video_id,trending_date,title_x,channel_title,category_id,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,kind,etag,items,title_y
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,SHANtell martin,748374,57527,2966,15954,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",People & Blogs
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/G9LHzQmx44rX2S5ya...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Comedy
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40944,BZt0qjTWNhw,18.14.06,The Cat Who Caught the Laser,AaronsAnimals,15,"aarons animals|""aarons""|""animals""|""cat""|""cats""...",1685609,38160,1385,2657,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Pets & Animals
40945,1h7KV2sjUWY,18.14.06,True Facts : Ant Mutualism,zefrank1,22,[none],1064798,60008,382,3936,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",People & Blogs
40946,D6Oy4LfoqsU,18.14.06,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,24,I gave safiya nygaard a perfect hair makeover ...,1066451,48068,1032,3992,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment
40947,oV0zkMe1K8s,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,1,"Black Panther|""HISHE""|""Marvel""|""Infinity War""|...",5660813,192957,2846,13088,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Film & Animation


In [23]:

combined_USdata.head()
    

,video_id,trending_date,title_x,channel_title,category_id,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,kind,etag,items,title_y
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,SHANtell martin,748374,57527,2966,15954,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",People & Blogs
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/G9LHzQmx44rX2S5ya...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Comedy
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment


In [24]:
print(combined_USdata)

          video_id trending_date  \
0      2kyS6SvSYSE      17.14.11   
1      1ZAPwfrtAFY      17.14.11   
2      5qpjK5DgCt4      17.14.11   
3      puqaWrEC7tY      17.14.11   
4      d380meD0W0M      17.14.11   
...            ...           ...   
40944  BZt0qjTWNhw      18.14.06   
40945  1h7KV2sjUWY      18.14.06   
40946  D6Oy4LfoqsU      18.14.06   
40947  oV0zkMe1K8s      18.14.06   
40948  ooyjaVdt-jA      18.14.06   

                                                 title_x  \
0                     WE WANT TO TALK ABOUT OUR MARRIAGE   
1      The Trump Presidency: Last Week Tonight with J...   
2      Racist Superman | Rudy Mancuso, King Bach & Le...   
3                       Nickelback Lyrics: Real or Fake?   
4                               I Dare You: GOING BALD!?   
...                                                  ...   
40944                       The Cat Who Caught the Laser   
40945                         True Facts : Ant Mutualism   
40946  I GAVE SAFIYA NYGAAR

In [25]:
combined_USdata["country"] = "US"
display(combined_USdata)

,video_id,trending_date,title_x,channel_title,category_id,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,kind,etag,items,title_y,country
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,SHANtell martin,748374,57527,2966,15954,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",People & Blogs,US
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment,US
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/G9LHzQmx44rX2S5ya...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Comedy,US
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment,US
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40944,BZt0qjTWNhw,18.14.06,The Cat Who Caught the Laser,AaronsAnimals,15,"aarons animals|""aarons""|""animals""|""cat""|""cats""...",1685609,38160,1385,2657,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Pets & Animals,US
40945,1h7KV2sjUWY,18.14.06,True Facts : Ant Mutualism,zefrank1,22,[none],1064798,60008,382,3936,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",People & Blogs,US
40946,D6Oy4LfoqsU,18.14.06,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,24,I gave safiya nygaard a perfect hair makeover ...,1066451,48068,1032,3992,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment,US
40947,oV0zkMe1K8s,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,1,"Black Panther|""HISHE""|""Marvel""|""Infinity War""|...",5660813,192957,2846,13088,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Film & Animation,US


In [26]:
renamed_combined_USdata = combined_USdata.rename(columns={"title_x": "video_title",
                                                          "title_y": "category_title"})
renamed_combined_USdata

,video_id,trending_date,video_title,channel_title,category_id,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,kind,etag,items,category_title,country
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,SHANtell martin,748374,57527,2966,15954,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",People & Blogs,US
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment,US
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/G9LHzQmx44rX2S5ya...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Comedy,US
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment,US
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40944,BZt0qjTWNhw,18.14.06,The Cat Who Caught the Laser,AaronsAnimals,15,"aarons animals|""aarons""|""animals""|""cat""|""cats""...",1685609,38160,1385,2657,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Pets & Animals,US
40945,1h7KV2sjUWY,18.14.06,True Facts : Ant Mutualism,zefrank1,22,[none],1064798,60008,382,3936,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",People & Blogs,US
40946,D6Oy4LfoqsU,18.14.06,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,24,I gave safiya nygaard a perfect hair makeover ...,1066451,48068,1032,3992,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Entertainment,US
40947,oV0zkMe1K8s,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,1,"Black Panther|""HISHE""|""Marvel""|""Infinity War""|...",5660813,192957,2846,13088,False,False,False,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...","{'kind': 'youtube#videoCategory', 'etag': '""m2...",Film & Animation,US


In [29]:
renamed_combined_USdata.to_csv("Resources/cleanedUSdata.csv",index=False)